In [1]:
!pip install -e ../../

Obtaining file:///C:/Users/netxph/Projects/mlpouch
  Attempting uninstall: mlpouch
    Found existing installation: mlpouch 0.0.1
    Uninstalling mlpouch-0.0.1:
      Successfully uninstalled mlpouch-0.0.1
  Running setup.py develop for mlpouch


In [2]:
import pandas as pd
from mlpouch.decomposition import FunkSVD
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
movies = pd.read_csv("data/movies_clean.csv")
del movies['Unnamed: 0']
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews = pd.read_csv("data/reviews_clean.csv")
del reviews['Unnamed: 0']
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
def create_train_test(X, order_by, training_size, testing_size, sample_size=10000):

    X = X.sort_values(by=order_by)
    sample = X.head(sample_size)
    
    X_train = sample.head(training_size)
    X_test = sample.tail(testing_size)
    
    return X_train, X_test

In [6]:
reviews_train, reviews_sub = create_train_test(reviews, 'date', 8000, 2000)

In [7]:
reviews_sub = reviews_train[["user_id", "movie_id", "rating"]]
print(reviews_sub.shape)
reviews_sub.head()

(8000, 3)


,user_id,movie_id,rating
498923,37287,2171847,6
442554,33140,444778,8
81920,6338,1411238,6
584570,43691,1496422,7
450669,33799,118799,5


In [8]:
movie_reviews = reviews_sub.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
movie_reviews.head()

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dec = FunkSVD(iters=20, learning_rate=0.005)
movie_reviews_pred = dec.fit_transform(movie_reviews)

DEBUG:root:Optimization Statistics
DEBUG:root:Iterations | Mean Squared Error 
DEBUG:root:1 		 32.52925492227926
DEBUG:root:2 		 21.045315068049756
DEBUG:root:3 		 15.636212675490565
DEBUG:root:4 		 12.174287188260323
DEBUG:root:5 		 9.745408881792981
DEBUG:root:6 		 7.955529361024876
DEBUG:root:7 		 6.582263671727173
DEBUG:root:8 		 5.496130394101053
DEBUG:root:9 		 4.61867453249143
DEBUG:root:10 		 3.8999047676257144
DEBUG:root:11 		 3.3062513850030606
DEBUG:root:12 		 2.8138672399886353
DEBUG:root:13 		 2.4048015734907744
DEBUG:root:14 		 2.064831812798343
DEBUG:root:15 		 1.7822734934183764
DEBUG:root:16 		 1.547340894915312
DEBUG:root:17 		 1.3517905141473172
DEBUG:root:18 		 1.1886941200935974
DEBUG:root:19 		 1.0522647704325074
DEBUG:root:20 		 0.9377041489896921


In [10]:
movie_reviews_pred

array([[4.77306485, 7.12535174, 3.05108399, ..., 7.3440012 , 6.12510732,
        5.88578137],
       [4.69428427, 5.69381839, 2.63089795, ..., 7.25727038, 7.81438206,
        5.60635846],
       [3.90677385, 5.71000676, 2.42428405, ..., 5.99572594, 5.17206254,
        4.79993664],
       ...,
       [3.91105266, 5.19775655, 2.4828121 , ..., 6.82473006, 6.61018196,
        5.20529041],
       [5.37844361, 7.13938873, 3.25149128, ..., 8.15854783, 7.9625299 ,
        6.39743805],
       [3.22412403, 4.92258198, 2.0629869 , ..., 5.411378  , 4.43231376,
        4.29807123]])

In [11]:
from mlpouch.similarity import DotProduct
import numpy as np

In [12]:
movies_mat = movies.iloc[:, 4:]
movies_mat.head()

,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
dp = DotProduct()
movies_dot = dp.fit_transform(movies_mat)
movies_dot

array([[3, 3, 3, ..., 0, 0, 0],
       [3, 3, 3, ..., 0, 0, 0],
       [3, 3, 3, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 2, 1],
       [0, 0, 0, ..., 2, 2, 1],
       [0, 0, 0, ..., 1, 1, 2]], dtype=int64)